In [1]:
# Steps
#1.Data collection
#2.Data processing
#3.Model training

In [2]:
# import libraries
import pandas as pd
import numpy as np

In [3]:
laptops = pd.read_csv("Dataset/laptops.csv")

In [4]:
laptops.head(3)

,laptop_id,img_link,name,price,processor,ram,os,storage,display,rating,no_of_ratings,no_of_reviews,laptop_brand,os_brand,processor_brand,usecases
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Intel Core i5 Processor (11th Gen),16 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,15.6,4.5,14.0,1.0,Lenovo,Windows,Intel,"Work and Productivity, Entertainment and Multi..."
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Intel Core i3 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,1 TB HDD|256 GB SSD,15.6,4.4,53.0,3.0,Lenovo,Windows,Intel,"Work and Productivity, Communication and Socia..."
2,2,https://rukminim1.flixcart.com/image/312/312/l...,ASUS TUF Gaming F15 Core i5 10th Gen,49990,Intel Core i5 Processor (10th Gen),8 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,15.6,4.4,4733.0,463.0,ASUS,Windows,Intel,"Work and Productivity, Communication and Socia..."


In [5]:
laptops.shape

(984, 16)

In [6]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   laptop_id        984 non-null    int64  
 1   img_link         984 non-null    object 
 2   name             984 non-null    object 
 3   price            984 non-null    int64  
 4   processor        984 non-null    object 
 5   ram              984 non-null    object 
 6   os               984 non-null    object 
 7   storage          984 non-null    object 
 8   display          984 non-null    float64
 9   rating           688 non-null    float64
 10  no_of_ratings    688 non-null    float64
 11  no_of_reviews    688 non-null    float64
 12  laptop_brand     984 non-null    object 
 13  os_brand         984 non-null    object 
 14  processor_brand  984 non-null    object 
 15  usecases         984 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 123.1+ KB


In [7]:
laptops = laptops[['laptop_id','img_link','name','price','processor','ram','os','storage','rating','laptop_brand','os_brand','processor_brand','usecases']]

In [8]:
laptops.head(3)

,laptop_id,img_link,name,price,processor,ram,os,storage,rating,laptop_brand,os_brand,processor_brand,usecases
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Intel Core i5 Processor (11th Gen),16 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,4.5,Lenovo,Windows,Intel,"Work and Productivity, Entertainment and Multi..."
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Intel Core i3 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,1 TB HDD|256 GB SSD,4.4,Lenovo,Windows,Intel,"Work and Productivity, Communication and Socia..."
2,2,https://rukminim1.flixcart.com/image/312/312/l...,ASUS TUF Gaming F15 Core i5 10th Gen,49990,Intel Core i5 Processor (10th Gen),8 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,4.4,ASUS,Windows,Intel,"Work and Productivity, Communication and Socia..."


In [9]:
# MISSING VALUES

In [10]:
laptops.isnull().sum()

laptop_id            0
img_link             0
name                 0
price                0
processor            0
ram                  0
os                   0
storage              0
rating             296
laptop_brand         0
os_brand             0
processor_brand      0
usecases             0
dtype: int64

In [11]:
laptops.dropna(inplace=True)

In [12]:
laptops.duplicated().sum()

0

In [13]:
laptops.iloc[0].laptop_brand

'Lenovo'

In [14]:
"""
Work and Productivity
Education and Learning
Communication and Social Networking
Entertainment and Multimedia
Creative Work
"""

'\nWork and Productivity\nEducation and Learning\nCommunication and Social Networking\nEntertainment and Multimedia\nCreative Work\n'

In [15]:
laptops.head(1)

,laptop_id,img_link,name,price,processor,ram,os,storage,rating,laptop_brand,os_brand,processor_brand,usecases
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Intel Core i5 Processor (11th Gen),16 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,4.5,Lenovo,Windows,Intel,"Work and Productivity, Entertainment and Multi..."


In [16]:
laptops['processor'] = laptops['processor'].apply(lambda x:x.split())
laptops['ram'] = laptops['ram'].apply(lambda x:x.split())
laptops['os'] = laptops['os'].apply(lambda x:x.split())
laptops['use'] = laptops['usecases'].apply(lambda x:x.split())

In [17]:
laptops['tags'] = laptops['processor']+laptops['ram']+laptops['os']+laptops['use']

In [18]:
laptops.head(1)

,laptop_id,img_link,name,price,processor,ram,os,storage,rating,laptop_brand,os_brand,processor_brand,usecases,use,tags
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,"[Intel, Core, i5, Processor, (11th, Gen)]","[16, GB, DDR4, RAM]","[Windows, 11, Operating, System]",512 GB SSD,4.5,Lenovo,Windows,Intel,"Work and Productivity, Entertainment and Multi...","[Work, and, Productivity,, Entertainment, and,...","[Intel, Core, i5, Processor, (11th, Gen), 16, ..."


In [19]:
laptops.iloc[0].tags

['Intel',
 'Core',
 'i5',
 'Processor',
 '(11th',
 'Gen)',
 '16',
 'GB',
 'DDR4',
 'RAM',
 'Windows',
 '11',
 'Operating',
 'System',
 'Work',
 'and',
 'Productivity,',
 'Entertainment',
 'and',
 'Multimedia,',
 'Creative',
 'Work']

In [20]:
new = laptops.drop(columns=['processor','ram','os','storage','rating','os_brand','processor_brand','use'])

In [21]:
new.head(2)

,laptop_id,img_link,name,price,laptop_brand,usecases,tags
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Lenovo,"Work and Productivity, Entertainment and Multi...","[Intel, Core, i5, Processor, (11th, Gen), 16, ..."
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Lenovo,"Work and Productivity, Communication and Socia...","[Intel, Core, i3, Processor, (11th, Gen), 8, G..."


In [22]:
new['tags'] = new['tags'].apply(lambda x:" ".join(x))

In [23]:
new.iloc[0].tags

'Intel Core i5 Processor (11th Gen) 16 GB DDR4 RAM Windows 11 Operating System Work and Productivity, Entertainment and Multimedia, Creative Work'

In [24]:
new['tags'] = new['tags'].apply(lambda x:x.lower())

In [25]:
new.head(2)

,laptop_id,img_link,name,price,laptop_brand,usecases,tags
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Lenovo,"Work and Productivity, Entertainment and Multi...",intel core i5 processor (11th gen) 16 gb ddr4 ...
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Lenovo,"Work and Productivity, Communication and Socia...",intel core i3 processor (11th gen) 8 gb ddr4 r...


In [26]:
new.iloc[0].tags

'intel core i5 processor (11th gen) 16 gb ddr4 ram windows 11 operating system work and productivity, entertainment and multimedia, creative work'

In [27]:
import nltk

In [28]:
from nltk.stem.porter import PorterStemmer

In [29]:
ps = PorterStemmer()

In [30]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [31]:
new['tags'] = new['tags'].apply(stem)

In [32]:
new.head(2)

,laptop_id,img_link,name,price,laptop_brand,usecases,tags
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Lenovo,"Work and Productivity, Entertainment and Multi...",intel core i5 processor (11th gen) 16 gb ddr4 ...
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Lenovo,"Work and Productivity, Communication and Socia...",intel core i3 processor (11th gen) 8 gb ddr4 r...


In [33]:
# counter vectorizing

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
cv = CountVectorizer(max_features = 5000, stop_words='english')

In [36]:
vector = cv.fit_transform(new['tags']).toarray()

In [37]:
vector

array([[0, 0, 1, ..., 0, 1, 2],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]], dtype=int64)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
similarity = cosine_similarity(vector)

In [40]:
similarity

array([[1.        , 0.70087664, 0.74095857, ..., 0.38188131, 0.6000992 ,
        0.70087664],
       [0.70087664, 1.        , 0.83462233, ..., 0.3441236 , 0.74560114,
        1.        ],
       [0.74095857, 0.83462233, 1.        , ..., 0.36380344, 0.78824078,
        0.83462233],
       ...,
       [0.38188131, 0.3441236 , 0.36380344, ..., 1.        , 0.375     ,
        0.3441236 ],
       [0.6000992 , 0.74560114, 0.78824078, ..., 0.375     , 1.        ,
        0.74560114],
       [0.70087664, 1.        , 0.83462233, ..., 0.3441236 , 0.74560114,
        1.        ]])

In [41]:
similarity[0]

array([1.        , 0.70087664, 0.74095857, 0.65081403, 0.54470478,
       0.46656947, 0.63510735, 0.70087664, 0.79388419, 0.56577895,
       0.91003151, 0.70087664, 0.95238095, 0.74095857, 0.65081403,
       0.70087664, 0.79091158, 0.56577895, 0.86452993, 0.46656947,
       0.65081403, 0.75093926, 0.56577895, 0.70087664, 0.68803296,
       0.56577895, 0.70087664, 0.56577895, 0.56577895, 0.70087664,
       1.        , 0.74438737, 0.79091158, 0.70087664, 0.70087664,
       0.56343617, 0.56577895, 0.65081403, 0.75093926, 0.65081403,
       0.81902836, 0.91003151, 0.70087664, 0.6000992 , 0.70087664,
       0.56577895, 0.56577895, 0.91003151, 0.70087664, 0.65081403,
       0.70087664, 0.75093926, 0.80952381, 1.        , 0.56577895,
       0.79091158, 0.70087664, 0.65081403, 0.74095857, 0.56577895,
       0.75093926, 0.46656947, 0.56577895, 0.70087664, 0.95553309,
       0.56577895, 0.68803296, 0.70087664, 0.52489066, 0.52489066,
       1.        , 0.65081403, 0.79091158, 0.70087664, 0.60075

In [42]:
len(similarity[0])

688

In [43]:
#Recommend

In [44]:
def recommend(laptop):
    index = new[new['usecases']==laptop].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key = lambda x:x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].usecases)

In [45]:
def recommend(use):
    index = new[new['usecases'] == use].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    printed_names = set()
    for i in distances[1:50]:
        row_index = i[0]
        name = new.iloc[row_index]['name']
        if name not in printed_names:
            printed_names.add(name)
            print(name)

In [46]:
sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x:x[1])

[(0, 1.0000000000000004),
 (30, 1.0000000000000004),
 (53, 1.0000000000000004),
 (70, 1.0000000000000004),
 (89, 1.0000000000000004),
 (107, 1.0000000000000004),
 (132, 1.0000000000000004),
 (154, 1.0000000000000004),
 (173, 1.0000000000000004),
 (197, 1.0000000000000004),
 (220, 1.0000000000000004),
 (236, 1.0000000000000004),
 (250, 1.0000000000000004),
 (267, 1.0000000000000004),
 (286, 1.0000000000000004),
 (302, 1.0000000000000004),
 (314, 1.0000000000000004),
 (327, 1.0000000000000004),
 (345, 1.0000000000000004),
 (355, 1.0000000000000004),
 (376, 1.0000000000000004),
 (393, 1.0000000000000004),
 (407, 1.0000000000000004),
 (423, 1.0000000000000004),
 (437, 1.0000000000000004),
 (449, 1.0000000000000004),
 (462, 1.0000000000000004),
 (476, 1.0000000000000004),
 (490, 1.0000000000000004),
 (506, 1.0000000000000004),
 (525, 1.0000000000000004),
 (545, 1.0000000000000004),
 (564, 1.0000000000000004),
 (577, 1.0000000000000004),
 (592, 1.0000000000000004),
 (604, 1.0000000000000004)

In [47]:
recommend("Work and Productivity, Entertainment and Multimedia, Creative Work")

Lenovo Intel Core i5 11th Gen
ASUS VivoBook K15 OLED (2022) Core i5 11th Gen
Lenovo IdeaPad Core i5 11th Gen
DELL Core i5 11th Gen
DELL Inspiron Core i5 11th Gen
HP Pavilion Intel Core i5 11th Gen
GIGABYTE G5 GD Core i5 11th Gen


In [48]:
import pickle

In [49]:
pickle.dump(new, open('model/usecases_list.pkl','wb'))

In [50]:
pickle.dump(similarity, open('model/similarity.pkl','wb'))

In [1]:
pip show pandas


Name: pandas
Version: 2.0.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: 
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distrib